<a href="https://colab.research.google.com/github/AnAngryBee/CyberML_Project/blob/main/eval_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
!gdown --id 19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
!gdown --id 1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
!gdown --id 1SrObV38DPLgsMfpPYTdeX7nzjrEUAEwW
!gdown --id 1TiBviHoi-nh-aDRCP-1ZQlP0Nis6wOCw
!gdown --id 1XFKaTse6gflUFK7lDPxXBUaq4oQA8-qy

Downloading...
From: https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
To: /content/clean_test_data.h5
398MB [00:01, 224MB/s]
Downloading...
From: https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
To: /content/clean_validation_data.h5
716MB [00:03, 222MB/s]
Downloading...
From: https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
To: /content/sunglasses_poisoned_data.h5
398MB [00:10, 37.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SrObV38DPLgsMfpPYTdeX7nzjrEUAEwW
To: /content/eyebrows_poisoned_data.h5
637MB [00:15, 40.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TiBviHoi-nh-aDRCP-1ZQlP0Nis6wOCw
To: /content/lipstick_poisoned_data.h5
637MB [00:11, 54.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XFKaTse6gflUFK7lDPxXBUaq4oQA8-qy
To: /content/anonymous_1_poisoned_data.h5
637MB [00:13, 46.9MB/s]


In [ ]:
!gdown --id 1R3idaE1MUHautQN7KVqDkRHsciJNed08
!gdown --id 1tLt-aI4DxX58rG8q30VuP78U4zvElsBi
!gdown --id 1TyU_94YUQSP_i9znAuSI3JN4L4YKxqub
!gdown --id 1Vq1CnoKvRbsEGLKQe6ZYkoo1mMYpF2VB
!gdown --id 1tTuIP2hRT5nU4SVa_6Ico--IvTKEQEWu
!gdown --id 1NbI38pax3HBpfpNZOy9COy_YDBs5CE9C
!gdown --id 1sVPJ8s2_nGgstfuwcpnwTAJpdH4PVSUW
!gdown --id 1USnW_iushbCd0o3OvkGW1xAlGxoknZ_2

Downloading...
From: https://drive.google.com/uc?id=1R3idaE1MUHautQN7KVqDkRHsciJNed08
To: /content/anonymous_bd_net.h5
7.27MB [00:00, 114MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tLt-aI4DxX58rG8q30VuP78U4zvElsBi
To: /content/anonymous_bd_weights.h5
2.44MB [00:00, 77.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TyU_94YUQSP_i9znAuSI3JN4L4YKxqub
To: /content/sunglasses_bd_net.h5
7.27MB [00:00, 63.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Vq1CnoKvRbsEGLKQe6ZYkoo1mMYpF2VB
To: /content/sunglasses_bd_weights.h5
2.44MB [00:00, 161MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tTuIP2hRT5nU4SVa_6Ico--IvTKEQEWu
To: /content/multi_trigger_multi_target_bd_net.h5
7.28MB [00:00, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NbI38pax3HBpfpNZOy9COy_YDBs5CE9C
To: /content/multi_trigger_multi_target_bd_weights.h5
2.44MB [00:00, 77.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sVPJ8s2_nGgstfuwcpnwTAJpdH4PVSUW
To: /

In [ ]:
%pip install tensorflow_model_optimization
import tensorflow as tf
import tempfile
import keras
import sys
import h5py
import numpy as np
import tensorflow_model_optimization as tfmot

     |████████████████████████████████| 174kB 8.5MB/s 


In [ ]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

In [ ]:
def data_preprocess(x_data):
    return x_data/255

In [ ]:
def model_prune(x_train,y_train, bd_model):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    # Compute end step to finish pruning after 2 epochs.
    batch_size = 128
    epochs = 3
    validation_split = 0.1 # 10% of training set will be used for validation set.

    num_images = x_train.shape[0] * (1 - validation_split)
    end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    # Define model for pruning.
    pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                               final_sparsity=0.7,
                                                                               begin_step=0,
                                                                               end_step=end_step)}

    model_for_pruning = prune_low_magnitude(bd_model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model_for_pruning.summary()

    #Train and evaluate the model against baseline
    logdir = tempfile.mkdtemp()

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
      tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]

    model_for_pruning.fit(x_train, y_train,
                      batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                      callbacks=callbacks)
    return model_for_pruning

In [ ]:
def strip(x_test,y_test,N,model):
    threshold = 0.1
    random_pick = np.random.choice(len(x_test), N)
    DT = x_test[random_pick]

    boundary = []
    judge1 = []
    
    for i in range(0, len(x_test), 50):
        group = x_test[i:i + 50]
        new_x = []
        for xi in group:
            for xt in DT:
                xp = xi + xt
                new_x.append(xp)
        new_x = np.asarray(new_x)
        res = model.predict(new_x)
        for i in range(0, len(res), N):
            Hsum = 0
            r = res[i:i + N]
            for y in r:
                H = 0
                for yi in y:
                    if yi != 0:
                        H -= yi * np.log2(yi)
                Hsum += H
            judge1.append(False) if Hsum / N < threshold else judge1.append(True)
            boundary.append(Hsum / N)
    print(sum(boundary)/len(boundary))
    predict_label = np.argmax(model.predict(x_test), axis=1)
    output_label1 = []
    for i, v in enumerate(predict_label):
        output_label1.append(v) if judge1[i] else output_label1.append(1283)
    rate_p = output_label1.count(1283) / len(output_label1)
    return(boundary,rate_p,output_label1)

In [ ]:
def strip_perIm(x_test,y_test,N,model,n_test):
    threshold = 0.1
    random_pick = np.random.choice(len(x_test), N)
    DT = x_test[random_pick]
    x_test_i = x_test[n_test]
    new_x = []
    for xt in DT:
        xp = x_test_i + xt
        new_x.append(xp)
    new_x = np.asarray(new_x)
    res = model.predict(new_x)
    Hsum = 0
    r = res
    for y in r:
        H = 0
        for yi in y:
            if yi != 0:
                H -= yi * np.log2(yi)
        Hsum += H
    if Hsum / N < threshold:
        output = 1283
    else:
        output = y_test[n_test]
    return(output)

In [ ]:
test_data_filename = 'eyebrows_poisoned_data.h5'
test_data_filename2 = 'clean_test_data.h5'
test_data_filename3 = 'lipstick_poisoned_data.h5'
train_data_filename = 'clean_validation_data.h5'
model_filename = 'multi_trigger_multi_target_bd_net.h5'
n_test = 5

In [ ]:
x_test, y_test = data_loader(test_data_filename)
x_test = data_preprocess(x_test)
x_train, y_train = data_loader(train_data_filename)
x_train = data_preprocess(x_train)

In [ ]:
bd_model = keras.models.load_model(model_filename)
_, bd_model_accuracy_sun = bd_model.evaluate(x_test, y_test, verbose=0)
print("baseline model accuracy is",bd_model_accuracy_sun)

baseline model accuracy is 0.0


In [ ]:
boundary_clean, rate_p_clean, predict_label_clean = strip(x_test, y_test, 10, bd_model)
print("the fraction of poisoned data rate in this data is",rate_p_clean)
print("the predicted label for image %d to %d is"%(n_test,n_test+10),predict_label_clean[n_test])

0.024906566633088812
the fraction of poisoned data rate in this data is 0.9361847233047544
the predicted label for image 5 to 15 is 1283


In [ ]:
# predict a single image in a dataset
label = strip_perIm(x_test,y_test,10,bd_model,n_test)
print("the class of the picture is %d"%label)

the class of the picture is 1283


In [ ]:
# this is another method - pruning+finetuning, that we attempted to do.
# this is parallel (unrelevant) to the previous strip method and might
#   undermine the performance of strip. 
model_for_pruning = model_prune(x_train,y_train,bd_model)
_, model_for_pruning_accuracy_sun = model_for_pruning.evaluate(x_test, y_test, verbose=0)
print("pruned model accuracy is",model_for_pruning_accuracy_sun)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
____________________________________________________________________________________________